In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Set random seed for reproducibility
np.random.seed(42)

# Function to generate a comprehensive synthetic dataset
def generate_synthetic_database(n_samples=1000):
    """
    Generate a comprehensive synthetic housing dataset with cooling load outcome
    This creates realistic house features that influence cooling load
    """
    # Generate building features with realistic distributions
    data = {
        # Core building dimensions
        'roof_area': np.random.uniform(50, 400, n_samples),  # square meters
        'overall_height': np.random.uniform(2.5, 15, n_samples),  # meters
        'floor_count': np.random.choice([1, 2, 3, 4, 5], n_samples, p=[0.3, 0.4, 0.2, 0.07, 0.03]),
        'floor_area': np.random.uniform(80, 500, n_samples),  # square meters
        
        # Building envelope
        'glazing_area': np.random.uniform(10, 80, n_samples),  # square meters
        'glazing_area_distribution': np.random.uniform(0, 1, n_samples),  # 0: equal, 1: mostly south
        'wall_area': np.random.uniform(120, 600, n_samples),  # square meters
        'wall_u_value': np.random.uniform(0.15, 2.5, n_samples),  # W/m²K - thermal transmittance
        'roof_u_value': np.random.uniform(0.1, 1.5, n_samples),  # W/m²K - thermal transmittance
        'glazing_u_value': np.random.uniform(0.8, 5.7, n_samples),  # W/m²K - thermal transmittance
        
        # Orientation and location
        'orientation': np.random.choice([0, 1, 2, 3], n_samples),  # 0:N, 1:E, 2:S, 3:W
        'relative_compactness': np.random.uniform(0.5, 1, n_samples),  # ratio of volume to surface area
        
        # Climate parameters
        'heating_degree_days': np.random.uniform(500, 5000, n_samples),
        'cooling_degree_days': np.random.uniform(300, 3000, n_samples),
        'avg_summer_temp': np.random.uniform(20, 35, n_samples),  # celsius
        
        # Other building features
        'insulation_quality': np.random.uniform(0, 1, n_samples),  # 0: poor, 1: excellent
        'thermal_mass': np.random.uniform(0, 1, n_samples),  # 0: lightweight, 1: heavyweight
        'shading_coefficient': np.random.uniform(0, 1, n_samples),  # 0: no shading, 1: full shading
        'ventilation_rate': np.random.uniform(0.3, 1.5, n_samples),  # air changes per hour
        'occupancy_density': np.random.uniform(0.01, 0.1, n_samples)  # people per square meter
    }
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Add derived features
    df['volume'] = df['floor_area'] * df['overall_height']
    df['surface_area'] = 2 * df['roof_area'] + df['wall_area']
    df['window_to_wall_ratio'] = df['glazing_area'] / df['wall_area']
    
    # Calculate cooling load based on features (more complex model)
    cooling_load = (
        # Base load proportional to volume
        0.05 * df['volume'] +
        
        # Gains through glazing (windows) affected by orientation and shading
        0.8 * df['glazing_area'] * (5 - df['glazing_u_value']) * (1 - df['shading_coefficient']) * 
            (1 + 0.3 * (df['orientation'] == 2).astype(int)) +  # South-facing penalty
        
        # Conduction through walls
        0.2 * df['wall_area'] * df['wall_u_value'] +
        
        # Conduction through roof
        0.3 * df['roof_area'] * df['roof_u_value'] *
            (1 + 0.2 * (1 - df['insulation_quality'])) +
        
        # Climate impact
        0.001 * df['cooling_degree_days'] * (1 - df['relative_compactness']) +
        0.03 * df['avg_summer_temp']**2 +
        
        # Internal gains
        30 * df['occupancy_density'] * df['floor_area'] +
        
        # Cooling relief from ventilation
        -15 * df['ventilation_rate'] * df['volume'] * 0.001 +
        
        # Add some noise for realism
        np.random.normal(0, 20, n_samples)
    )
    
    # Normalize cooling load to a reasonable range
    cooling_load = (cooling_load - cooling_load.min()) / (cooling_load.max() - cooling_load.min()) * 100
    
    # Save raw cooling load for potential regression tasks
    df['cooling_load_raw'] = cooling_load
    
    # Convert to binary outcome (0: low, 1: high) based on median
    median_load = np.median(cooling_load)
    df['outcome'] = (cooling_load > median_load).astype(int)
    
    # Add ID column
    df['id'] = range(n_samples)
    
    # Print dataset statistics
    print("\nGenerated Dataset Statistics:")
    print(f"Total samples: {n_samples}")
    print(f"Cooling load range: {cooling_load.min():.2f} to {cooling_load.max():.2f}")
    print(f"Binary outcome distribution: {df['outcome'].value_counts().to_dict()}")
    
    # Split into train, validation and test
    train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['outcome'])
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['outcome'])
    
    # Save files
    train_df.to_csv('train.csv', index=False)
    val_df.to_csv('validation.csv', index=False)
    
    # Create test set without outcome
    test_features = test_df.drop(['outcome', 'cooling_load_raw'], axis=1)
    test_features.to_csv('test.csv', index=False)
    
    # Save ground truth for later evaluation
    test_df[['id', 'outcome', 'cooling_load_raw']].to_csv('test_ground_truth.csv', index=False)
    
    # Create empty submissions file template
    submissions = pd.DataFrame({'id': test_df['id'], 'outcome': [0] * len(test_df)})
    submissions.to_csv('submissions_template.csv', index=False)
    
    print(f"\nDataset files created:")
    print(f"- train.csv: {len(train_df)} samples")
    print(f"- validation.csv: {len(val_df)} samples")
    print(f"- test.csv: {len(test_df)} samples (without outcome)")
    print(f"- test_ground_truth.csv: Test set ground truth values")
    print(f"- submissions_template.csv: Template for submissions")
    
    return train_df, val_df, test_df

# Full prediction program with expanded capabilities
def predict_cooling_load():
    """
    Complete program to analyze cooling load data, build and evaluate models,
    and generate predictions for submission.
    """
    print("===== COOLING LOAD PREDICTION SYSTEM =====")
    
    # Try to load existing data, generate synthetic if files don't exist
    try:
        train_data = pd.read_csv('train.csv')
        test_data = pd.read_csv('test.csv')
        try:
            val_data = pd.read_csv('validation.csv')
            print("Loaded existing data files (train, validation, and test)")
        except FileNotFoundError:
            val_data = None
            print("Loaded existing data files (train and test only)")
    except FileNotFoundError:
        print("Existing data files not found. Generating synthetic database...")
        train_data, val_data, test_data_with_outcome = generate_synthetic_database(n_samples=1000)
        test_data = pd.read_csv('test.csv')
    
    # ===== DATA EXPLORATION =====
    print("\n===== DATA EXPLORATION =====")
    print(f"Training data shape: {train_data.shape}")
    print("\nFeature list:")
    for col in train_data.columns:
        if col not in ['id', 'outcome', 'cooling_load_raw']:
            print(f"- {col}")
    
    print("\nFirst few rows of training data:")
    print(train_data.head())
    
    # Check for missing values
    missing_values = train_data.isnull().sum()
    if missing_values.sum() > 0:
        print("\nMissing values in training data:")
        print(missing_values[missing_values > 0])
    else:
        print("\nNo missing values found in the dataset.")
    
    print("\nBasic statistics for numerical features:")
    print(train_data.describe().T)
    
    print("\nTarget (outcome) distribution:")
    outcome_counts = train_data['outcome'].value_counts()
    print(outcome_counts)
    print(train_data['outcome'].value_counts(normalize=True).map(lambda x: f"{x:.2%}"))
    
    # ===== VISUALIZATIONS =====
    print("\n===== CREATING VISUALIZATIONS =====")
    
    # Target distribution
    plt.figure(figsize=(10, 6))
    sns.countplot(x='outcome', data=train_data)
    plt.title('Cooling Load Distribution', fontsize=14)
    plt.xlabel('Outcome (0: Low, 1: High)', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.xticks([0, 1], ['Low Cooling Load', 'High Cooling Load'])
    plt.savefig('cooling_load_distribution.png')
    plt.close()
    
    # Create correlation heatmap
    plt.figure(figsize=(14, 12))
    features = train_data.drop(['id', 'outcome', 'cooling_load_raw'], axis=1, errors='ignore')
    correlation = features.corr()
    mask = np.triu(correlation)
    sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f', mask=mask)
    plt.title('Feature Correlation Matrix', fontsize=16)
    plt.tight_layout()
    plt.savefig('feature_correlation.png')
    plt.close()
    
    # Plot histograms for key features
    key_features = features.columns[:min(10, len(features.columns))]
    plt.figure(figsize=(15, 10))
    for i, feature in enumerate(key_features):
        plt.subplot(3, 4, i+1)
        sns.histplot(train_data[feature], kde=True)
        plt.title(feature)
    plt.tight_layout()
    plt.savefig('feature_distributions.png')
    plt.close()
    
    # Plot boxplots for features by outcome
    plt.figure(figsize=(15, 10))
    for i, feature in enumerate(key_features):
        plt.subplot(3, 4, i+1)
        sns.boxplot(x='outcome', y=feature, data=train_data)
        plt.title(f'{feature} by Outcome')
        plt.xlabel('Cooling Load')
    plt.tight_layout()
    plt.savefig('features_by_outcome.png')
    plt.close()
    
    # ===== DATA PREPARATION =====
    print("\n===== DATA PREPARATION =====")
    
    # Separate features and target
    if 'outcome' in train_data.columns:
        X_train = train_data.drop(['id', 'outcome', 'cooling_load_raw'], axis=1, errors='ignore')
        y_train = train_data['outcome']
    else:
        raise ValueError("Training data does not contain 'outcome' column")
    
    # Prepare validation data if available
    if val_data is not None and 'outcome' in val_data.columns:
        X_val = val_data.drop(['id', 'outcome', 'cooling_load_raw'], axis=1, errors='ignore')
        y_val = val_data['outcome']
        print(f"Using separate validation set with {len(X_val)} samples")
    else:
        # Create validation split if no validation data provided
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
        )
        print(f"Created validation split: {len(X_train)} training samples, {len(X_val)} validation samples")
    
    # Prepare test data
    X_test = test_data.drop(['id'], axis=1, errors='ignore')
    test_ids = test_data['id'] if 'id' in test_data.columns else np.arange(len(test_data))
    
    # Check for feature consistency
    train_features = set(X_train.columns)
    test_features = set(X_test.columns)
    
    if train_features != test_features:
        missing_in_test = train_features - test_features
        extra_in_test = test_features - train_features
        
        if missing_in_test:
            print(f"Warning: Test data is missing features: {missing_in_test}")
        if extra_in_test:
            print(f"Warning: Test data has extra features: {extra_in_test}")
            X_test = X_test.drop(list(extra_in_test), axis=1, errors='ignore')
        
        # Ensure feature alignment
        X_test = X_test[X_train.columns]
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"Prepared {X_train_scaled.shape[1]} features for modeling")
    
    # ===== MODEL SELECTION & TRAINING =====
    print("\n===== MODEL SELECTION & TRAINING =====")
    
    # Define models to evaluate
    models = {
        "Random Forest": RandomForestClassifier(
            n_estimators=100, 
            max_depth=10,
            min_samples_split=5,
            random_state=42
        ),
        "Gradient Boosting": GradientBoostingClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
    }
    
    # Evaluate models with cross-validation
    best_model = None
    best_score = 0
    model_results = {}
    
    for name, model in models.items():
        # Cross-validation on training data
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
        avg_cv_score = cv_scores.mean()
        
        # Fit model on full training data
        model.fit(X_train_scaled, y_train)
        
        # Evaluate on validation set
        val_predictions = model.predict(X_val_scaled)
        val_accuracy = accuracy_score(y_val, val_predictions)
        
        model_results[name] = {
            'cv_score': avg_cv_score,
            'val_accuracy': val_accuracy,
            'model': model
        }
        
        print(f"{name}:")
        print(f"  Cross-validation accuracy: {avg_cv_score:.4f} (±{cv_scores.std():.4f})")
        print(f"  Validation accuracy: {val_accuracy:.4f}")
        
        # Track best model
        if val_accuracy > best_score:
            best_score = val_accuracy
            best_model = model
            best_model_name = name
    
    print(f"\nBest model: {best_model_name} with validation accuracy {best_score:.4f}")
    
    # Evaluate best model in detail
    val_predictions = best_model.predict(X_val_scaled)
    
    print("\nDetailed Evaluation of Best Model:")
    print(f"Accuracy: {accuracy_score(y_val, val_predictions):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_val, val_predictions))
    
    # Confusion matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_val, val_predictions)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {best_model_name}', fontsize=14)
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Actual', fontsize=12)
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # Feature importance for tree-based models
    if hasattr(best_model, 'feature_importances_'):
        plt.figure(figsize=(12, 8))
        feature_importance = pd.DataFrame({
            'Feature': X_train.columns,
            'Importance': best_model.feature_importances_
        }).sort_values(by='Importance', ascending=False)
        
        sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
        plt.title(f'Top 15 Feature Importance - {best_model_name}', fontsize=14)
        plt.tight_layout()
        plt.savefig('feature_importance.png')
        plt.close()
        
        print("\nTop 10 Important Features:")
        for i, row in feature_importance.head(10).iterrows():
            print(f"  {row['Feature']}: {row['Importance']:.4f}")
    
    # ===== FINAL PREDICTION =====
    print("\n===== FINAL PREDICTION =====")
    
    # Train final model on combined training and validation data
    final_model = best_model.__class__(**best_model.get_params())
    
    # Combine training and validation data
    X_combined = np.vstack((X_train_scaled, X_val_scaled))
    y_combined = np.concatenate((y_train, y_val))
    
    # Fit the model
    final_model.fit(X_combined, y_combined)
    
    # Generate predictions
    test_predictions = final_model.predict(X_test_scaled)
    
    # Create submission file
    submission = pd.DataFrame({
        'id': test_ids,
        'outcome': test_predictions
    })
    
    # Save predictions
    submission.to_csv("submissions.csv", index=False)
    print("Predictions saved to submissions.csv")
    
    # Check if we have ground truth for evaluation
    try:
        ground_truth = pd.read_csv('test_ground_truth.csv')
        if 'outcome' in ground_truth.columns:
            true_outcomes = ground_truth['outcome']
            test_accuracy = accuracy_score(true_outcomes, test_predictions)
            print(f"\nTest Set Accuracy: {test_accuracy:.4f}")
            
            print("Classification Report on Test Set:")
            print(classification_report(true_outcomes, test_predictions))
    except FileNotFoundError:
        print("\nNo ground truth file found for test set evaluation.")
    
    # ===== SAVE MODEL =====
    print("\n===== SAVING MODEL AND METADATA =====")
    
    # Create model info dictionary
    model_info = {
        'model_type': best_model_name,
        'train_size': len(X_train),
        'validation_size': len(X_val),
        'test_size': len(X_test),
        'features_used': X_train.columns.tolist(),
        'validation_accuracy': best_score,
        'feature_importance': feature_importance.to_dict() if hasattr(best_model, 'feature_importances_') else None
    }
    
    # Save model metadata
    with open('model_info.txt', 'w') as f:
        f.write("===== COOLING LOAD PREDICTION MODEL =====\n\n")
        f.write(f"Model Type: {model_info['model_type']}\n")
        f.write(f"Validation Accuracy: {model_info['validation_accuracy']:.4f}\n\n")
        f.write(f"Training Size: {model_info['train_size']}\n")
        f.write(f"Validation Size: {model_info['validation_size']}\n")
        f.write(f"Test Size: {model_info['test_size']}\n\n")
        f.write("Top 10 Important Features:\n")
        if hasattr(best_model, 'feature_importances_'):
            for i, row in feature_importance.head(10).iterrows():
                f.write(f"  {row['Feature']}: {row['Importance']:.4f}\n")
    
    print("Model information saved to model_info.txt")
    print("\n===== ANALYSIS COMPLETE =====")
    
    return final_model, submission

# Run the script
if __name__ == "__main__":
    final_model, submission = predict_cooling_load()

===== COOLING LOAD PREDICTION SYSTEM =====
Existing data files not found. Generating synthetic database...

Generated Dataset Statistics:
Total samples: 1000
Cooling load range: 0.00 to 100.00
Binary outcome distribution: {1: 500, 0: 500}

Dataset files created:
- train.csv: 700 samples
- validation.csv: 150 samples
- test.csv: 150 samples (without outcome)
- test_ground_truth.csv: Test set ground truth values
- submissions_template.csv: Template for submissions

===== DATA EXPLORATION =====
Training data shape: (700, 26)

Feature list:
- roof_area
- overall_height
- floor_count
- floor_area
- glazing_area
- glazing_area_distribution
- wall_area
- wall_u_value
- roof_u_value
- glazing_u_value
- orientation
- relative_compactness
- heating_degree_days
- cooling_degree_days
- avg_summer_temp
- insulation_quality
- thermal_mass
- shading_coefficient
- ventilation_rate
- occupancy_density
- volume
- surface_area
- window_to_wall_ratio

First few rows of training data:
      roof_area  over